In [ ]:
%matplotlib inline  
%reload_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

import numpy as np
import pandas as pd
# import seaborn as sns

from access_literature_data import medline
from access_science_shared import standardizer

import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('./../src/')
import nar171107f_citation_network as cit
import resci_tools as ret

In [ ]:
save_images = False
save_tables = True

In [ ]:
# get full citation network of research publications linked to pubmed
pmid_2_pmid = cit.gene_pmid_research_gene_pmid_research()

In [ ]:
# get publication years of pubmed IDs
df_m = medline.select_medline_wos_records(
    columns_sql='''
        medline.pubmed_id,
        medline.pubdate_year''',
    taxon_id='all',
    kind='research',
    unambiguous=True
)

In [ ]:
# pubmed, and whether pmid has a human and/or not-human
gene2pubmed = medline.gene2pubmed(taxon_id='all')
taxon_pubmed = gene2pubmed.copy()
taxon_pubmed.loc[:, 'has_human'] = taxon_pubmed['taxon_ncbi'] == 9606
taxon_pubmed.loc[:, 'has_not_human'] = taxon_pubmed['taxon_ncbi'] != 9606
taxon_pubmed = taxon_pubmed[['pubmed_id', 'has_human', 'has_not_human']].drop_duplicates()

In [ ]:
taxon_id = 9606
ref_genes = standardizer.reference_genes(taxon_id, 'orp')  # offical nomenclature, liteature, protein-coding
ref_gene2pubmed = medline.gene2pubmed(taxon_id, ['pubmed_id', 'gene_ncbi'], paper_kind='research')

In [ ]:
# get year of initial report
import nar170902_discovery_year as yr
initial_year = yr.get_year_of_discovery(taxon_id, ref_genes).reset_index()
initial = initial_year[['gene_ncbi', 'first_year']].rename(columns={'first_year': 'pubdate_year'})
initial.loc[:, 'initial_report'] = True

In [ ]:
df_m_human_gene = pd.merge(df_m, ref_gene2pubmed)

In [ ]:
df_m_human_gene = pd.merge(
    df_m_human_gene,
    initial,
    left_on=['pubdate_year', 'gene_ncbi'],
    right_on=['pubdate_year', 'gene_ncbi'],
    how='left'
)

In [ ]:
df_m_human_gene['initial_report'] = df_m_human_gene['initial_report'].fillna(False)

In [ ]:
df_m_human = df_m_human_gene[['pubmed_id', 'pubdate_year', 'initial_report']].drop_duplicates()

In [ ]:
df_m_human = df_m_human.groupby(['pubmed_id', 'pubdate_year']).agg(any).reset_index()

In [ ]:
master = pd.merge(pmid_2_pmid, df_m_human_gene, left_on='citing', right_on='pubmed_id').drop('pubmed_id', 1)

In [ ]:
master = pd.merge(
    master,
    taxon_pubmed,
    left_on='cited',
    right_on='pubmed_id',
    how='left'
)

In [ ]:
m = master[['citing', 'initial_report', 'has_human', 'has_not_human']].groupby(
    ['citing']).agg(sum).reset_index()

In [ ]:
m['excess_not_human'] =  m['has_not_human'] > m['has_human']

In [ ]:
a = pd.merge(
    (m.set_index('citing') >0).reset_index(),
    master[['citing', 'pubdate_year']].drop_duplicates())

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(
    x='pubdate_year',
    y='excess_not_human',
    data=a[a['pubdate_year'].isin(np.arange(1980, 2001))],
    hue='initial_report'
)

if save_images:
    ret.export_image('171107_citations_between_organisms/more_non_human_by_year.pdf')

In [ ]:
sns.barplot(x='initial_report', y='excess_not_human',data=a)

if save_images:
    ret.export_image('171107_citations_between_organisms/more_non_human.pdf')

In [ ]:
out = a[['citing', 'initial_report', 'excess_not_human', 'pubdate_year']].rename(columns={'citing': 'pubmed_id'})

if save_tables:
    ret.export_full_frame(
        '171107_citations_between_organisms/data.csv',
        out,
        save_index=False       
    )